In [ ]:
import pickle as pkl
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

import spacy
  
nlp = spacy.load('en_core_web_lg')
from sklearn.metrics.pairwise import cosine_similarity

import itertools

# Get Data

## Changes from year to year

In [ ]:
changes = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0'])

changes.head()

In [ ]:
industries = changes[['gvkey','naics']].drop_duplicates()
industries['naics'] = industries.naics.astype('str').str.slice(0,2)

## SC Linking Table for 2010s

In [ ]:
# this does a little bit of a test on the reporting requirements. 
# number 

'''c_linksTest = pd.read_csv("../../data/companyData/compustatSCLinked.csv")[['srcdate','gvkey','cgvkey']]
c_linksTest['year'] = c_linksTest.srcdate.astype('str').str.slice(0,4).astype('int64')

bs = c_linksTest[c_linksTest.year < 2014]
print("Customers per supplier, 1978-2013 Pd: ", len(bs.cgvkey.unique())/len(bs.gvkey.unique()))

bs2 = c_linksTest[c_linksTest.year > 2010]
print("Customers per supplier, Recent Pd: ", len(bs2.cgvkey.unique())/len(bs2.gvkey.unique()))'''


In [ ]:
c_links = pd.read_csv("../../data/companyData/compustatSCLinked.csv")

c_links['year'] = c_links.srcdate.astype('str').str.slice(0,4).astype('int64')

c_links = c_links[c_links.year > 2009][['year','gvkey','cgvkey','salecs']].\
    rename(columns = {'cgvkey': 'customer_gvkey','gvkey': 'supplier_gvkey'})


print(c_links.shape)

c_links.head()

In [ ]:
industries.columns = ['customer_gvkey','customer_naics']

c_links = c_links.merge(industries)

industries.columns = ['supplier_gvkey','supplier_naics']

c_links = c_links.merge(industries)
c_links.head()

In [ ]:
c_links.shape

Now see if it's common to have one in and one out of the industries of interest. 

For now, let's keep all the different industry types.

We can always filter later if we need to.

In [ ]:
'''ofInterest = ['11','21','22','23','31','32','33','42','44','45','48','49']

c_linksCut = c_links[~(c_links.customer_naics.isin(ofInterest) & c_links.supplier_naics.isin(ofInterest))]

c_linksCut['relat'] = c_linksCut.customer_naics + "_" + c_linksCut.supplier_naics
c_linksCut.relat.value_counts()[0:10]

c_linksCut.supplier_naics.value_counts()[0:10]'''

## Compustat and ABI Linking

In [ ]:
gvKey_abiLinkingTable = pd.read_csv('../../data/companyData/linkingTable.csv').drop(columns = ['Unnamed: 0'])

base_columns = gvKey_abiLinkingTable.columns 
customer_columns = "customer_" + base_columns
supplier_columns = "supplier_" + base_columns


gvKey_abiLinkingTable.head()

In [ ]:
hasMatch = gvKey_abiLinkingTable.gvkey.unique()

In [ ]:
sum(c_links.supplier_gvkey.isin(hasMatch) | c_links.customer_gvkey.isin(hasMatch))

# Merge

In [ ]:

#########################
# merge in customer information
gvKey_abiLinkingTable.columns = customer_columns

print(c_links.shape)
c_linksMerge1 = c_links.merge(gvKey_abiLinkingTable, on ='customer_gvkey')
print(c_links.shape,c_linksMerge1.shape)



#########################
# and merge in supplier 
gvKey_abiLinkingTable.columns = supplier_columns

print(c_links.shape)
c_linksMerge2 = c_linksMerge1.merge(gvKey_abiLinkingTable, on ='supplier_gvkey')
print(c_links.shape,c_linksMerge2.shape)

In [ ]:
c_linksMerge2.to_csv("../../data/companyData/clinks_IG_selected.csv")

In [ ]:
gvKey_abiLinkingTable

This is probably because: (1) companies are not in North America, or (2) companies are not in the physical goods industries we're interested in. We can verify this though: look at c_links where both the customer and supplier are in the dataset of interest.

In [ ]:
chq     = pd.read_csv("../../data/chq.csv",dtype={'cstatZipcode': 'object'}).drop(columns = {'Unnamed: 0'})

c_linkTest = c_links[c_links.customer_gvkey.isin(chq.gvkey.unique()) & \
                     c_links.supplier_gvkey.isin(chq.gvkey.unique())]

print("Percent of firms with a match: ", c_linksMerge2.shape[0]/c_linkTest.shape[0])

It's entirely possible that we have too small of a sample from the 2010s alone. Let's just try it though and see how it goes.

First, make a sample with the companies on three years of either side of when it reports another customer.

In [ ]:
scTableCustomers = c_linksMerge2.copy()[['year','customer_gvkey','customer_abi']].drop_duplicates()
scTableSuppliers = c_linksMerge2.copy()[['year','supplier_gvkey','supplier_abi']].drop_duplicates()

In [ ]:
scTableCustomers.head()

In [ ]:
def makeThreeEitherSide(df): 
    yrPlus1 = df.copy(); yrPlus1['year'] += 1
    yrPlus2 = df.copy(); yrPlus2['year'] += 1
    yrPlus3 = df.copy(); yrPlus3['year'] += 1
    
    yrMinus1 = df.copy(); yrMinus1['year'] -= 1
    yrMinus2 = df.copy(); yrMinus2['year'] -= 1
    yrMinus3 = df.copy(); yrMinus3['year'] -= 1
    
    all = pd.concat([yrPlus1,yrPlus2,yrPlus3,yrMinus1,yrMinus2,yrMinus3])
    
    return(all)

In [ ]:
allCustomerData = makeThreeEitherSide(scTableCustomers)
allCustomerData.columns = ['year','gvkey','abi']


allSupplierData = makeThreeEitherSide(scTableSuppliers)
allSupplierData.columns = ['year','gvkey','abi']

In [ ]:
allSupplierData.year

In [ ]:
allAbi = allCustomerData.abi.append(allSupplierData.abi).drop_duplicates()

In [ ]:
hqsOnly = pd.read_csv("../../data/ig2010s_uniqueHQs.csv").drop(columns = {'Unnamed: 0'})

In [ ]:
hqsOnly.head()

In [ ]:
hq = pd.read_csv("../../data/ig2010s_uniqueHQs_multLocations.csv").\
    drop(columns = {'Unnamed: 0'}).\
    rename(columns = {'archive_version_year': 'year'})

hq['year'] = hq.year.astype('int64')

hqRelevant = hq[hq.abi.isin(allAbi)]

In [ ]:
hqRelevant[hqRelevant.abi == 71340]

In [ ]:
allSupplierData = allSupplierData.merge(hqRelevant).drop_duplicates()
allCustomerData = allCustomerData.merge(hqRelevant).drop_duplicates()

In [ ]:
allSupplierData.head()

In [ ]:
allCustomerData.to_csv("../../data/companyData/allCustomerData.csv")
allSupplierData.to_csv("../../data/companyData/allSupplierData.csv")

# Find Customer and Supplier pairings and merge with change data

In [ ]:
allSupplierData = pd.read_csv("../../data/companyData/allSupplierData.csv").drop(columns = ['Unnamed: 0'])
allCustomerData = pd.read_csv("../../data/companyData/allCustomerData.csv").drop(columns = ['Unnamed: 0'])

In [ ]:
changes = pd.read_csv("../../data/companyData/compustatChanges_2010s.csv").drop(columns = ['Unnamed: 0'])
changes.head()

In [ ]:
changes.year.value_counts()

In [ ]:
allSupplierData.head()

In [ ]:
allSupplierData.zipcode.value_counts()[-75:-50]

In [ ]:
suppliers = changes.merge(allSupplierData[['year','gvkey','zipcode']])
suppliers.shape

In [ ]:
customers = changes.merge(allCustomerData[['year','gvkey','zipcode']])
customers.head()

In [ ]:
print(allCustomerData.shape,allSupplierData.shape)

In [ ]:
# allWeather = pd.read_csv("../../../../../../../Volumes/backup2/dissData/prism/allWeatherBins_2010.2019.csv").\

allWeather = pd.read_csv("../../data/companyData/allWeatherBins_2010.2019.csv").\
    drop(columns = {"Unnamed: 0",'zip','yearQuarter'})

In [ ]:
allWeather['yearQtr'] = allWeather.year + (allWeather.qtr - 1)/4

lag1 = allWeather.copy()[['zipcode','yearQtr','precip_quant_1.0','tmax_quant_1.0']]
lag1['yearQtr'] += 0.25
lag1.rename(columns = {'precip_quant_1.0':'lag1_precip_quant_1.0','tmax_quant_1.0':'lag1_tmax_quant_1.0'},inplace = True)
# allWeather = allWeather.merge(lag1)


lag2 = allWeather.copy()[['zipcode','yearQtr','precip_quant_1.0','tmax_quant_1.0']]
lag2['yearQtr'] += 0.5
lag2.rename(columns = {'precip_quant_1.0':'lag2_precip_quant_2.0','tmax_quant_1.0':'lag2_tmax_quant_2.0'},inplace = True)
# allWeather = allWeather.merge(lag2)


lag3 = allWeather.copy()[['zipcode','yearQtr','precip_quant_1.0','tmax_quant_1.0']]
lag3['yearQtr'] += 0.75
lag3.rename(columns = {'precip_quant_1.0':'lag3_precip_quant_3.0','tmax_quant_1.0':'lag3_tmax_quant_3.0'},inplace = True)
# allWeather = allWeather.merge(lag3)


lag4 = allWeather.copy()[['zipcode','yearQtr','precip_quant_1.0','tmax_quant_1.0']]
lag4['yearQtr'] += 1
lag4.rename(columns = {'precip_quant_1.0':'lag4_precip_quant_4.0','tmax_quant_1.0':'lag4_tmax_quant_4.0'},inplace = True)
# allWeather = allWeather.merge(lag4)

print(allWeather.shape)

In [ ]:
allWeather.year.value_counts()

In [ ]:
allWeather_withLags = allWeather.merge(lag1).merge(lag2).merge(lag3).merge(lag4)

print(oneMerge.shape)

allWeather_withLags.year.value_counts()

In [ ]:
customersWithWeather = customers.merge(allWeather_withLags)
customersWithWeather.shape

In [ ]:
suppliersWithWeather = suppliers.merge(allWeather_withLags)
suppliersWithWeather.shape

In [ ]:
suppliersWithWeather.to_csv("../../data/companyData/suppliersWithWeather.csv")
customersWithWeather.to_csv("../../data/companyData/customersWithWeather.csv")

In [ ]:
suppliersWithWeather = pd.read_csv("../../data/companyData/suppliersWithWeather.csv").drop(columns = {'Unnamed: 0'})
customersWithWeather = pd.read_csv("../../data/companyData/customersWithWeather.csv").drop(columns = {'Unnamed: 0'})

In [ ]:
customersWithWeather.columns

In [ ]:
customersWithWeather.head()

In [ ]:
frames = [customersWithWeather, suppliersWithWeather]

allCompanies = pd.concat(frames).drop_duplicates()

print(allCompanies.shape)

In [ ]:
allCompanies.head()

In [ ]:
allCompanies.to_csv("../../data/companyData/allCompaniesWithWeather.csv")

In [ ]:
allCompanies.head()

## Get first-hop SC data

In [ ]:
c_links = pd.read_csv("../../data/companyData/clinks_IG_selected.csv").drop(columns = {'Unnamed: 0'})
c_links.head()

In [ ]:
c_links['suppliers'] = 1
custExp = c_links[['year', 'customer_gvkey', 'salecs','suppliers']].groupby(['year','customer_gvkey']).sum().\
    reset_index().rename(columns = {'salecs': 'totalExp'})

custExp.head()



In [ ]:
print("Number of firms with no exp information and multiple suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 1))
print("Number of firms with no exp information and >5 suppliers: ", \
          sum(custExp[custExp.totalExp == 0].suppliers > 5))


Most of these firms have expenditure information. We can look at:
    - Expenditure-weighted (just do equal shares if no exp information)
    - Largest supplier

In [ ]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()
print(customerDB.shape)

customerDB.head()

## Biggest Supplier
Focus on weather of biggest supplier.

First find the max by supplier. Add back in any rows with only 1 supplier.

In [ ]:
# https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-value-in-groups-using-groupby
idx = customerDB.groupby(['year','gvkey']).salecs.\
    transform(max) == customerDB.salecs
largestSuppliers = customerDB[idx].reset_index(drop = True)
print(largestSuppliers.shape)

# find companies who only have one other supplier
singleSuppliers = customerDB[customerDB.suppliers == 1].reset_index(drop = True)
print(singleSuppliers.shape)

# find largest suppliers of different companies
largestSuppliers = largestSuppliers.append(singleSuppliers).drop_duplicates()
print(largestSuppliers.shape)



In [ ]:
largestSuppliers.head()

In [ ]:
suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].\
    rename(columns = {'gvkey': 'supplier_gvkey',
                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',
                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})

In [ ]:
largestSuppliersWithWeather = largestSuppliers.merge(customersWithWeather[['year', 'qtr', 'gvkey', 'companyName', 'naics',
       'revenueChange', 'costChange',
       'inventoryChange', 'zipcode', 'tmax_quant_1.0', 'precip_quant_1.0']]).merge(suppliers_toMerge)

largestSuppliersWithWeather.shape

In [ ]:
largestSuppliersWithWeather.head()

In [ ]:
largestSuppliersWithWeather.to_csv("../../data/companyData/largestSuppliersWithWeather.csv")

## Sales-Weighted Average
If a company doesn't have sales-specific information, then assume equal shares. This doesn't happen for too many of the companies, thankfully.

In [ ]:
customerDB = c_links[['year','customer_gvkey','supplier_gvkey','salecs']].\
    merge(custExp).rename(columns = {'customer_gvkey': 'gvkey'}).drop_duplicates()

customerDB['salesWeight'] = customerDB.salecs/customerDB.totalExp

customerDB.fillna(1, inplace = True)

Now merge this with the supplier weather data, and use the sales weights to find a sales-weighted average of the weather conditions for the suppliers.

In [ ]:
suppliers_toMerge = suppliersWithWeather[['year','qtr','gvkey','tmax_quant_1.0','precip_quant_1.0']].\
    rename(columns = {'gvkey': 'supplier_gvkey',
                      'tmax_quant_1.0': 'supplier_tmax_quant_1.0',
                      'precip_quant_1.0': 'supplier_precip_quant_1.0'})

In [ ]:
supplierWeather = customerDB.merge(suppliers_toMerge)[['year','qtr','gvkey','salesWeight','supplier_tmax_quant_1.0','supplier_precip_quant_1.0']]

supplierWeather['supplier_tmax_quant_1.0']   = supplierWeather.salesWeight*supplierWeather['supplier_tmax_quant_1.0']
supplierWeather['supplier_precip_quant_1.0'] = supplierWeather.salesWeight*supplierWeather['supplier_precip_quant_1.0']

supplierWeather = supplierWeather[['year','qtr','gvkey','supplier_tmax_quant_1.0','supplier_precip_quant_1.0']]

In [ ]:
supplierWtdAvgWeather = supplierWeather.groupby(['year','qtr','gvkey']).sum().reset_index()

In [ ]:
supplierWtdAvgWeather.shape

Merge the supplier weighted average weather data with the customer data that has weather as well.

In [ ]:
wtdAvgSuppliers = customersWithWeather[['year', 'qtr', 'gvkey', 'companyName', 'naics',
       'revenueChange', 'costChange',
       'inventoryChange', 'zipcode', 'tmax_quant_1.0', 'precip_quant_1.0']].merge(supplierWtdAvgWeather)

wtdAvgSuppliers.head()

In [ ]:
wtdAvgSuppliers.columns

In [ ]:
wtdAvgSuppliers.to_csv("../../data/companyData/wtdAvgSuppliers.csv")